In [77]:
import pymc as pm
import numpy as np
import pandas as pd


In [78]:
reg_szn_df = pd.read_csv('./data/MRegularSeasonCompactResults.csv')
reg_szn_df = reg_szn_df[reg_szn_df['Season'] == 2023]
reg_szn_df['score_diff'] = reg_szn_df['WScore'] - reg_szn_df['LScore']
reg_szn_df['days_till_tourney'] = 132 - reg_szn_df['DayNum']
reg_szn_df = reg_szn_df[['WTeamID', 'LTeamID', 'score_diff', 'days_till_tourney']]
reg_szn_df.columns = ['team1', 'team2', 'score_diff', 'days_till_tourney']

data = reg_szn_df.copy()


In [79]:
teams = list(set(data['team1']).union(data['team2']))
team_to_idx = {team: idx for idx, team in enumerate(teams)}
data['team1_idx'] = data['team1'].map(team_to_idx)
data['team2_idx'] = data['team2'].map(team_to_idx)

# Convert team names to indices
teams = list(set(data['team1']).union(data['team2']))
team_to_idx = {team: idx for idx, team in enumerate(teams)}
data['team1_idx'] = data['team1'].map(team_to_idx)
data['team2_idx'] = data['team2'].map(team_to_idx)

# Hyperparameters for recency weighting
tau_decay = 0.01  # Controls how fast weights decay with time

# Calculate weights for each game based on recency
data['weight'] = np.exp(-tau_decay * data['days_till_tourney'])

# Model
with pm.Model() as model:
    # Priors for team ratings (mean and variance)
    team_means = pm.Normal("team_means", mu=0, sigma=10, shape=len(teams))
    team_vars = pm.HalfNormal("team_vars", sigma=5, shape=len(teams))

    # Calculate game-specific ratings differences
    rating_diff = team_means[data['team1_idx']] - team_means[data['team2_idx']]

    # Point differences follow a normal distribution
    game_variances = team_vars[data['team1_idx']] + team_vars[data['team2_idx']]
    game_dist = pm.Normal.dist(mu=rating_diff, sigma=pm.math.sqrt(game_variances))

    # Weighted log-likelihood
    weighted_logp = pm.logp(game_dist, data['score_diff']) * data['weight']

    # Add the weighted likelihood to the model
    pm.Potential("weighted_likelihood", weighted_logp.sum())

    # Sampling
    trace = pm.sample(1000, 
                      chains = 4,
                      cores = 1,
                      return_inferencedata=True, 
                      progressbar=True,
                      compute_convergence_checks=False,  # Speeds up sampling for large models
                      nuts_sampler="numpyro",
                      nuts = {'target_accept': 0.7}
                      )

# Posterior mean and variance for each team
team_ratings = {
    team: {
        "mean": trace.posterior["team_means"].mean(dim=["chain", "draw"]).values[team_to_idx[team]],
        "variance": trace.posterior["team_vars"].mean(dim=["chain", "draw"]).values[team_to_idx[team]],
    }
    for team in teams
}

/home/reynald/anaconda3/envs/sprvs/lib/python3.12/site-packages/pymc/sampling/jax.py:470: UserWarning: There are not enough devices to run parallel chains: expected 4 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(4)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  pmap_numpyro = MCMC(
sample: 100%|██████████| 2000/2000 [01:42<00:00, 19.46it/s, 15 steps of size 2.72e-01. acc. prob=0.80]


In [80]:
ratings_df = pd.DataFrame(team_ratings).T
ratings_df.index.name = 'TeamID'
ratings_df = ratings_df.reset_index()

In [81]:
tourney_df = pd.read_csv('data/MNCAATourneyCompactResults.csv')
tourney_df = tourney_df[tourney_df['Season'] == 2023][['WTeamID', 'LTeamID', 'WScore', 'LScore']]
tourney_df['score_diff'] = tourney_df['WScore'] - tourney_df['LScore']
tourney_df = tourney_df[['WTeamID', 'LTeamID', 'score_diff']]

In [82]:
merged_df = pd.merge(tourney_df, ratings_df, left_on = 'WTeamID', right_on = 'TeamID')\
                .drop(columns = ['WTeamID', 'TeamID'])\
                .rename(columns = {'mean': 'w_mean', 'variance': 'w_var'})\
                .merge(ratings_df, left_on = 'LTeamID', right_on = 'TeamID')\
                .drop(columns = ['LTeamID', 'TeamID'])\
                .rename(columns = {'mean': 'l_mean', 'variance': 'l_var'})
merged_df['pred_score_diff'] = merged_df['w_mean'] - merged_df['l_mean']
merged_df['pred_score_var'] = merged_df['w_var'] + merged_df['l_var']
merged_df = merged_df[['score_diff', 'pred_score_diff', 'pred_score_var']]

In [83]:
from scipy.stats import norm as N

-np.mean(np.log(N.sf(0,loc = merged_df['pred_score_diff'], scale = np.sqrt(merged_df['pred_score_var']))))

0.8882396297572314

In [84]:
np.mean(N.sf(0,loc = merged_df['pred_score_diff'], scale = np.sqrt(merged_df['pred_score_var'])) > 0.5)

0.6119402985074627